## Check List 3.2 - Combined Detector (ArucoStero, MultiICP)
* **3.2.1 Combined Detector**  
  - initialize, set_config, get_config, get_image, detect, detect_and_register, disconnect   
  
  
* **TBD**
  - Auto initialization to estimate initial guess for ICP is not perfect
  - Robust and reliable initial guess for gloabl registration will be done
  - Multiple instance for the same class will be done

## Set running directory to Project source

In [1]:
import os
import sys
import numpy as np
import cv2
import copy
import matplotlib.pyplot as plt
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

## 3.2.1 Combined Detector

In [10]:
from pkg.detector.combined_detector import CombinedDetector
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

##### create cameras

In [5]:
kn = Kinect()
rs = RealSense()

##### create ArucoStereo instance

In [7]:
stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[kn, rs])
stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()

##### create multiICP instance

In [8]:
micp = MultiICP(rs)
micp.initialize()

In [11]:
detector = CombinedDetector([stereo, micp])

##### create SceneBuilder instance

In [12]:
scene_builder = SceneBuilder.instance(detector=detector)

##### set reference coordinate and viewpoint (by StereoAruco)

In [ ]:
# deprecated: scene_builder.reset_reference_coord(ref_name="floor")
T_rs = np.matmul(scene_builder.ref_coord_inv, stereo.T_c12)
viewpoint = gscene.create_safe(gtype=GEOTYPE.ARROW, name="viewpoint", link_name="base_link",
                               dims=(0.2, 0.04, 0.04), center=T_rs[:3,3], rpy=Rot2rpy(T_rs[:3,:3]),
                               color=(1, 0, 0, 0.3), display=True, fixed=True, collision=False)

##### create geometry scene

In [ ]:
xyz_rpy_robots = scene_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = scene_builder.create_gscene(crob)
gscene.show_pose(crob.home_pose)

### Set MultiICP configs
* You have to make micp, hrule and grule for each object you want to detect

* hrule means heuristic rule for special object which cannot be detected thorugh mask rcnn using COCO dataset

* grule means initial guess(R,t) for ICP

* Run shraed detector to detect object in color image

##### Run shared detector for object detection on bash
```bash
python3 /home/jhkim/Projects/rnb-planning/src/pkg/detector/multiICP/shared_detector.py --dims='(720,1280,3)'
```

#### Clearing shared detector channels if zombie memory remains
```python
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")
```

In [8]:
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")

from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(micp.img_dim+(3,))()
sd.init()

In [11]:
obj_info_dict = get_obj_info()

In [12]:
# object items you want to detect
# heuristic rule items, Initial guess rule items
micp_cup = MultiICP_Obj(obj_info_dict["cup"], None,
                        OffsetOnModelCoord("cup", R=Rot_axis(1, np.pi/2), offset=None))

In [13]:
micp_dict = {"cup": micp_cup}
micp.set_config(micp_dict, sd, crob, viewpoint)

##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ



In [20]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

name_mask is ['closet', 'bed']
===== Detected : bed, 1 object(s) =====

'bed' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
registration::RegistrationResult with fitness=8.409394e-01, inlier_rmse=6.899803e-02, and correspondence_set size of 5049
Access transformation to get result.
Transformation is:
[[ 0.82007189 -0.1025711   0.56299316 -0.58887756]
 [-0.22207067 -0.96374952  0.14788999  0.66678282]
 [ 0.52741516 -0.24630468 -0.81312192  4.61567619]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=8.305593e-01, inlier_rmse=4.570206e-02, and correspondence_set size of 2495
Access transformation to get result.
Transformation is:
[[ 0.82394519 -0.07580986  0.56157567 -0.58022322]
 [-0.17396279 -0.97699637  0.12334907  0.72283548]
 [ 0.53930633 -0.19932613 -0.81817955  4.62225097]
 [ 0.          0.          0.          1.        ]]
Found 6DoF pose of bed
===== Apply heuristic rule fo

##### disconnect()

In [19]:
detector.disconnect()